In [1]:
import pandas as pd
import plotly.express as px

In [2]:
stipends = pd.read_csv("data/boston_stipends.csv")

In [8]:
avg_by_year = stipends[["Academic Year", "University", "Overall Pay"]].groupby(["Academic Year", "University"]).mean().reset_index()

In [12]:
stipends["Academic Year"].value_counts()

Academic Year
2020.0    119
2016.0     92
2022.0     73
2018.0     67
2019.0     58
2021.0     50
2017.0     34
2023.0     31
2014.0     19
2015.0     15
2024.0     14
2013.0      5
2011.0      2
2012.0      2
2025.0      1
Name: count, dtype: int64

In [11]:
stipends_over_time = px.line(
    avg_by_year,
    x="Academic Year",
    y="Overall Pay",
    color="University"
).update_layout(yaxis_tickprefix = '$', yaxis_tickformat = ',.')
stipends_over_time.add_hline(y=63942)
stipends_over_time.add_hline(y=15650)
stipends_over_time.show()